In [1]:
! pip install watchdog

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 20;

<IPython.core.display.Javascript object>

In [3]:
from OutputProcessing import process_output_files

In [ ]:
process_output_files('../responses/', 1000)

IntProgress(value=0, description='Iter 0/1000', max=1000)

'Mean: 0.546'

,name,neg,neu,pos,compound
0,Page #6,0.922,0.714,0.590,0.983
1,Page #15,0.744,0.085,0.064,0.942
2,Page #12,0.383,0.791,0.332,0.930
3,Page #24,0.926,0.391,0.178,0.907
4,Page #0,0.850,0.539,0.615,0.892
5,Page #7,0.414,0.295,0.367,0.780
6,Page #8,0.947,0.518,0.980,0.758
7,Page #23,0.034,0.094,0.211,0.748
8,Page #9,0.849,0.691,0.722,0.721
9,Page #17,0.731,0.367,0.977,0.688
